In [1]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
import torch
import pandas as pd
from sklearn.model_selection import train_test_split

import os

# Отключаем W&B
os.environ["WANDB_DISABLED"] = "true"

# Подготовка данных
data = pd.DataFrame({
    'text': [
        "Текст 1: Сегодня погода солнечная и теплая. -- Текст 2: Планирую погулять в парке. Является продолжением? [да/нет]:", 
        "Текст 1: Вчера шел дождь, и мы остались дома. -- Текст 2: Завтра я планирую купить продукты. Является продолжением? [да/нет]:",
        "Текст 1: Он купил билеты на самолет. -- Текст 2: Он с нетерпением ждал своей поездки. Является продолжением? [да/нет]:", 
        "Текст 1: Мы были на концерте вчера вечером. -- Текст 2: Сегодня я снова пойду на концерт. Является продолжением? [да/нет]:",
        "Текст 1: Она сделала домашку за вечер. -- Текст 2: У нее не было времени на другие дела. Является продолжением? [да/нет]:", 
        "Текст 1: Я купил новую книгу по программированию. -- Текст 2: Я начал изучать новый язык программирования. Является продолжением? [да/нет]:",
        "Текст 1: Он выиграл соревнования по шахматам. -- Текст 2: Он готовится к следующему турниру. Является продолжением? [да/нет]:", 
        "Текст 1: Мы отпраздновали Новый год с друзьями. -- Текст 2: В следующем году мы планируем провести праздник вместе. Является продолжением? [да/нет]:", 
        "Текст 1: Я занимаюсь спортом каждое утро. -- Текст 2: Сегодня я пропустил тренировку. Является продолжением? [да/нет]:",
        "Текст 1: Вчера я потерял свой телефон. -- Текст 2: Сегодня мне удалось его найти. Является продолжением? [да/нет]:",
        "Текст 1: Мы поехали на отдых на море. -- Текст 2: Мы планируем вернуться туда летом. Является продолжением? [да/нет]:", 
        "Текст 1: Я начал читать новую книгу по истории. -- Текст 2: Эта книга рассказывает о древних цивилизациях. Является продолжением? [да/нет]:"
    ],
    'label': [1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0]  # 1 - продолжение, 0 - не продолжение
})

train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Загрузка токенизатора и модели
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3medium_based_on_gpt2")
model = GPT2ForSequenceClassification.from_pretrained("sberbank-ai/rugpt3medium_based_on_gpt2", num_labels=2)

# Токенизация
def tokenize_function(examples):
    return tokenizer(list(examples["text"]), padding="max_length", truncation=True, max_length=128)

train_encodings = tokenize_function(train_data)
val_encodings = tokenize_function(val_data)

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, log_loss
import numpy as np

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=-1)

    # Accuracy
    accuracy = accuracy_score(labels, preds)

    # Precision, Recall, F1
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="binary", zero_division=1
    )

    # ROC AUC (если метка 0/1)
    if len(np.unique(labels)) == 2:
        try:
            roc_auc = roc_auc_score(labels, pred.predictions[:, 1])  # Использует вероятности положительного класса
        except ValueError:
            roc_auc = None
    else:
        roc_auc = None
    
    # Логарифмическая потеря
    try:
        logloss = log_loss(labels, pred.predictions)
    except ValueError:
        logloss = None

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "roc_auc": roc_auc,
        "log_loss": logloss
    }


# Подготовка датасетов
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_encodings, train_data['label'].tolist())
val_dataset = Dataset(val_encodings, val_data['label'].tolist())

# Настройка тренировки
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    logging_steps=10,
    logging_first_step=True, 
    learning_rate=2e-5,
    per_device_train_batch_size=2,  
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    #report_to="none",
)

# Создание Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Тренировка
trainer.train()

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/rugpt3medium_based_on_gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Roc Auc,Log Loss,Runtime,Samples Per Second,Steps Per Second
1,0.717900,0.189350,1.000000,1.000000,1.000000,1.000000,None,None,0.123000,24.390000,16.260000
2,0.822900,0.066057,1.000000,1.000000,1.000000,1.000000,None,None,0.064700,46.369000,30.913000
3,0.822900,0.088495,1.000000,1.000000,1.000000,1.000000,None,None,0.069500,43.169000,28.779000


Trainer is attempting to log a value of "None" of type <class 'NoneType'> for key "eval/roc_auc" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "None" of type <class 'NoneType'> for key "eval/log_loss" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "None" of type <class 'NoneType'> for key "eval/roc_auc" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "None" of type <class 'NoneType'> for key "eval/log_loss" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "None" of type <class 'NoneType'> for key "eval/roc_auc" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrec

TrainOutput(global_step=15, training_loss=0.7925993283589681, metrics={'train_runtime': 10.9484, 'train_samples_per_second': 2.466, 'train_steps_per_second': 1.37, 'total_flos': 6268772155392.0, 'train_loss': 0.7925993283589681, 'epoch': 3.0})